## tensorflow入门

## mnist数据集的使用

In [4]:
# coding:utf-8
# 从tensorflow.examples.tutorials.mnist引入模块。这是TensorFlow为了教学MNIST而提前编制的程序
from tensorflow.examples.tutorials.mnist import input_data
# 从MNIST_data/中读取MNIST数据。这条语句在数据不存在时，会自动执行下载
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

# 查看训练数据的大小
print('训练数据的大小',mnist.train.images.shape)  # (55000, 784)
print('训练数据标签的大小',mnist.train.labels.shape)  # (55000, 10)

# 查看验证数据的大小
print('验证数据的大小',mnist.validation.images.shape)  # (5000, 784)
print('验证数据标签的大小',mnist.validation.labels.shape)  # (5000, 10)

# 查看测试数据的大小
print('测试数据的大小',mnist.test.images.shape)  # (10000, 784)
print('测试标签数据的大小',mnist.test.labels.shape)  # (10000, 10)

# 打印出第0幅图片的向量表示
print('打印第一张照片',mnist.train.images[0, :])

# 打印出第0幅图片的标签
print('打印第一张照片的标签',mnist.train.labels[0, :])

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
训练数据的大小 (55000, 784)
训练数据标签的大小 (55000, 10)
验证数据的大小 (5000, 784)
验证数据标签的大小 (5000, 10)
测试数据的大小 (10000, 784)
测试标签数据的大小 (10000, 10)
打印第一张照片 [0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.        

## 保存前20张图片

In [12]:
#coding: utf-8
from tensorflow.examples.tutorials.mnist import input_data
import scipy.misc
import os

# 读取MNIST数据集。如果不存在会事先下载。
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

# 我们把原始图片保存在MNIST_data/raw/文件夹下
# 如果没有这个文件夹会自动创建
save_dir = 'MNIST_data/raw/'
if os.path.exists(save_dir) is False:
    os.makedirs(save_dir)

# 保存前20张图片
for i in range(20):
    # 请注意，mnist.train.images[i, :]就表示第i张图片（序号从0开始）
    image_array = mnist.train.images[i, :]
    # TensorFlow中的MNIST图片是一个784维的向量，我们重新把它还原为28x28维的图像。
    image_array = image_array.reshape(28, 28)
    # 保存文件的格式为 mnist_train_0.jpg, mnist_train_1.jpg, ... ,mnist_train_19.jpg
    filename = save_dir + 'mnist_train_%d.jpg' % i
    # 将image_array保存为图片
    # 先用scipy.misc.toimage转换为图像，再调用save直接保存。
    
    #scipy.misc.toimage(image_array, cmin=0.0, cmax=1.0).save(filename)

print('Please check: %s ' % save_dir)



Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Please check: MNIST_data/raw/ 


## 简单的softmax分类

In [14]:
# coding:utf-8
# 导入tensorflow。
# 这句import tensorflow as tf是导入TensorFlow约定俗成的做法，请大家记住。
import tensorflow as tf
# 导入MNIST教学的模块
from tensorflow.examples.tutorials.mnist import input_data
# 与之前一样，读入MNIST数据
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

# 创建x，x是一个占位符（placeholder），代表待识别的图片
x = tf.placeholder(tf.float32, [None, 784])

# W是Softmax模型的参数，将一个784维的输入转换为一个10维的输出
# 在TensorFlow中，变量的参数用tf.Variable表示
W = tf.Variable(tf.zeros([784, 10]))
# b是又一个Softmax模型的参数，我们一般叫做“偏置项”（bias）。
b = tf.Variable(tf.zeros([10]))

# y=softmax(Wx + b)，y表示模型的输出
y = tf.nn.softmax(tf.matmul(x, W) + b)

# y_是实际的图像标签，同样以占位符表示。
y_ = tf.placeholder(tf.float32, [None, 10])

# 至此，我们得到了两个重要的Tensor：y和y_。
# y是模型的输出，y_是实际的图像标签，不要忘了y_是独热表示的
# 下面我们就会根据y和y_构造损失

# 根据y, y_构造交叉熵损失
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y)))

# 有了损失，我们就可以用随机梯度下降针对模型的参数（W和b）进行优化
train_step = tf.train.GradientDescentOptimizer(0.01).minimize(cross_entropy)

# 创建一个Session。只有在Session中才能运行优化步骤train_step。
sess = tf.InteractiveSession()
# 运行之前必须要初始化所有变量，分配内存。
tf.global_variables_initializer().run()
print('start training...')

# 进行1000步梯度下降
for _ in range(1000):
    # 在mnist.train中取100个训练数据
    # batch_xs是形状为(100, 784)的图像数据，batch_ys是形如(100, 10)的实际标签
    # batch_xs, batch_ys对应着两个占位符x和y_
    batch_xs, batch_ys = mnist.train.next_batch(100)
    # 在Session中运行train_step，运行时要传入占位符的值
    sess.run(train_step, feed_dict={x: batch_xs, y_: batch_ys})

# 正确的预测结果
correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
# 计算预测准确率，它们都是Tensor
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
# 在Session中运行Tensor可以得到Tensor的值
# 这里是获取最终模型的正确率
print(sess.run(accuracy, feed_dict={x: mnist.test.images, y_: mnist.test.labels}))  # 0.9185


Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


/home/liyuan3970/anaconda3/lib/python3.7/site-packages/tensorflow/python/client/session.py:1735: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


start training...
0.9128


## 卷积神经网络识别

In [ ]:
# coding: utf-8
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data


def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)


def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)


def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')


def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                          strides=[1, 2, 2, 1], padding='SAME')


if __name__ == '__main__':
    # 读入数据
    mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)
    # x为训练图像的占位符、y_为训练图像标签的占位符
    x = tf.placeholder(tf.float32, [None, 784])
    y_ = tf.placeholder(tf.float32, [None, 10])

    # 将单张图片从784维向量重新还原为28x28的矩阵图片
    x_image = tf.reshape(x, [-1, 28, 28, 1])

    # 第一层卷积层
    W_conv1 = weight_variable([5, 5, 1, 32])
    b_conv1 = bias_variable([32])
    h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
    h_pool1 = max_pool_2x2(h_conv1)

    # 第二层卷积层
    W_conv2 = weight_variable([5, 5, 32, 64])
    b_conv2 = bias_variable([64])
    h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
    h_pool2 = max_pool_2x2(h_conv2)

    # 全连接层，输出为1024维的向量
    W_fc1 = weight_variable([7 * 7 * 64, 1024])
    b_fc1 = bias_variable([1024])
    h_pool2_flat = tf.reshape(h_pool2, [-1, 7 * 7 * 64])
    h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)
    # 使用Dropout，keep_prob是一个占位符，训练时为0.5，测试时为1
        keep_prob = tf.placeholder(tf.float32)
        h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

    # 把1024维的向量转换成10维，对应10个类别，对应的生成一个10维的数据
    W_fc2 = weight_variable([1024, 10])
    b_fc2 = bias_variable([10])
    y_conv = tf.matmul(h_fc1_drop, W_fc2) + b_fc2

    # 我们不采用先Softmax再计算交叉熵的方法，而是直接用tf.nn.softmax_cross_entropy_with_logits直接计算
    cross_entropy = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y_conv))
    # 同样定义train_step
    train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

    # 定义测试的准确率
    correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y_, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

    # 创建Session和变量初始化
    sess = tf.InteractiveSession()
    sess.run(tf.global_variables_initializer())

    # 训练20000步
    for i in range(20000):
        batch = mnist.train.next_batch(50)
        # 每100步报告一次在验证集上的准确度
        if i % 100 == 0:
            train_accuracy = accuracy.eval(feed_dict={
                x: batch[0], y_: batch[1], keep_prob: 1.0})
            print("step %d, training accuracy %g" % (i, train_accuracy))
        sess.run(train_step,feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})   
       # train_step.run(feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})

    # 训练结束后报告在测试集上的准确度
    print("test accuracy %g" % accuracy.eval(feed_dict={
        x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0}))


W1004 22:31:00.366731 140168492443456 deprecation.py:323] From <ipython-input-1-132123ad7681>:27: read_data_sets (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
W1004 22:31:00.369883 140168492443456 deprecation.py:323] From /home/liyuan3970/anaconda3/lib/python3.7/site-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:260: maybe_download (from tensorflow.contrib.learn.python.learn.datasets.base) is deprecated and will be removed in a future version.
Instructions for updating:
Please write your own downloading logic.
W1004 22:31:00.372128 140168492443456 deprecation.py:323] From /home/liyuan3970/anaconda3/lib/python3.7/site-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:262: extract_images (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed

Extracting MNIST_data/train-images-idx3-ubyte.gz


W1004 22:31:00.715982 140168492443456 deprecation.py:323] From /home/liyuan3970/anaconda3/lib/python3.7/site-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:267: extract_labels (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use tf.data to implement this functionality.
W1004 22:31:00.737642 140168492443456 deprecation.py:323] From /home/liyuan3970/anaconda3/lib/python3.7/site-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:110: dense_to_one_hot (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use tf.one_hot on tensors.
W1004 22:31:00.845816 140168492443456 deprecation.py:323] From /home/liyuan3970/anaconda3/lib/python3.7/site-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:290: DataSet.__init__ (from tensorflow.contrib.learn.python.learn.

Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


W1004 22:31:01.333488 140168492443456 deprecation.py:506] From <ipython-input-1-132123ad7681>:54: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
W1004 22:31:01.363921 140168492443456 deprecation.py:323] From <ipython-input-1-132123ad7681>:63: softmax_cross_entropy_with_logits (from tensorflow.python.ops.nn_ops) is deprecated and will be removed in a future version.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



step 0, training accuracy 0.06
step 100, training accuracy 0.86


## 对交叉熵的理解

熵其实是信息量的期望值，它是一个随机变量的确定性的度量。熵越大，变量的取值越不确定，反之就越确定。


In [7]:
# coding: utf-8
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data


def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)


def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)


def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')


def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                          strides=[1, 2, 2, 1], padding='SAME')


if __name__ == '__main__':
    # 读入数据
    mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)
    # x为训练图像的占位符、y_为训练图像标签的占位符
    x = tf.placeholder(tf.float32, [None, 784])
    y_ = tf.placeholder(tf.float32, [None, 10])

    # 将单张图片从784维向量重新还原为28x28的矩阵图片
    x_image = tf.reshape(x, [-1, 28, 28, 1])

    # 第一层卷积层
    W_conv1 = weight_variable([5, 5, 1, 32])
    b_conv1 = bias_variable([32])
    h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
    h_pool1 = max_pool_2x2(h_conv1)

    # 第二层卷积层
    W_conv2 = weight_variable([5, 5, 32, 64])
    b_conv2 = bias_variable([64])
    h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
    h_pool2 = max_pool_2x2(h_conv2)

    # 全连接层，输出为1024维的向量
    W_fc1 = weight_variable([7 * 7 * 64, 1024])
    b_fc1 = bias_variable([1024])
    h_pool2_flat = tf.reshape(h_pool2, [-1, 7 * 7 * 64])
    h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)
    # 使用Dropout，keep_prob是一个占位符，训练时为0.5，测试时为1
    keep_prob = tf.placeholder(tf.float32)
    h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

    # 把1024维的向量转换成10维，对应10个类别，对应的生成一个10维的数据
    W_fc2 = weight_variable([1024, 10])
    b_fc2 = bias_variable([10])
    y_conv = tf.matmul(h_fc1_drop, W_fc2) + b_fc2

    # 我们不采用先Softmax再计算交叉熵的方法，而是直接用tf.nn.softmax_cross_entropy_with_logits直接计算
    cross_entropy = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y_conv))
    # 同样定义train_step
    train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

    # 定义测试的准确率
    correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y_, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

    # 创建Session和变量初始化

    with tf.Session() as sess:
            #sess = tf.InteractiveSession()
            sess.run(tf.global_variables_initializer())
            for i in range(20000):
                batch = mnist.train.next_batch(50)
#                 if i % 100 == 0:
#                     train_accuracy = accuracy.eval(feed_dict={x: batch[0], y_: batch[1], keep_prob: 1.0})
#                     print("step %d, training accuracy %g" % (i, train_accuracy))
                sess.run(train_step,feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})   
                
        

    # 训练20000步
#     for i in range(20000):
#         batch = mnist.train.next_batch(50)
#         # 每100步报告一次在验证集上的准确度
#         if i % 100 == 0:
#             train_accuracy = accuracy.eval(feed_dict={
#                 x: batch[0], y_: batch[1], keep_prob: 1.0}) 
#             print("step %d, training accuracy %g" % (i, train_accuracy))
#         sess.run(train_step,feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})   
       # train_step.run(feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})

#     # 训练结束后报告在测试集上的准确度
#     print("test accuracy %g" % accuracy.eval(feed_dict={
#         x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0}))


Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [2]:

import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data",one_hot=True)

batch_size = 100                #设置每一轮训练的batch大小
learning_rate = 0.8             #学习率
learning_rate_decay = 0.999     #学习率的衰减
max_steps = 30000               #最大训练步数

#定义存储训练轮数的变量，在使用Tensorflow训练神经网络时，
#一般会将代表训练轮数的变量通过trainable参数设置为不可训练的
training_step = tf.Variable(0,trainable=False)

#定义得到隐藏层和输出层的前向传播计算方式，激活函数使用relu()
def hidden_layer(input_tensor,weights1,biases1,weights2,biases2,layer_name):
    layer1=tf.nn.relu(tf.matmul(input_tensor,weights1)+biases1)
    return tf.matmul(layer1,weights2)+biases2

x = tf.placeholder(tf.float32,[None,784],name="x-input")   #INPUT_NODE=784
y_ = tf.placeholder(tf.float32,[None,10],name="y-output")   #OUT_PUT=10
#生成隐藏层参数，其中weights包含784x500=392000个参数
weights1=tf.Variable(tf.truncated_normal([784,500],stddev=0.1))
biases1 = tf.Variable(tf.constant(0.1,shape=[500]))
#生成输出层参数，其中weights2包含500x10=5000个参数
weights2 = tf.Variable(tf.truncated_normal([500, 10], stddev=0.1))
biases2 = tf.Variable(tf.constant(0.1, shape=[10]))

#计算经过神经网络前向传播后得到的y的值，这里没有使用滑动平均
y = hidden_layer(x,weights1,biases1,weights2,biases2,'y')


#初始化一个滑动平均类，衰减率为0.99
#为了使模型在训练前期可以更新地更快，这里提供了num_updates参数
#并设置为当前网络的训练轮数
averages_class = tf.train.ExponentialMovingAverage(0.99,training_step)
#定义一个更新变量滑动平均值的操作需要向滑动平均类的apply()函数提供一个参数列表
#train_variables()函数返回集合图上Graph.TRAINABLE_VARIABLES中的元素，
#这个集合的元素就是所有没有指定trainable_variables=False的参数
averages_op = averages_class.apply(tf.trainable_variables())
#再次计算经过神经网络前向传播后得到的y的值，这里使用了滑动平均，但要牢记滑动平均值只是一个影子变量
average_y = hidden_layer(x,averages_class.average(weights1),averages_class.average(biases1),
                averages_class.average(weights2),averages_class.average(biases2),'average_y')

#计算交叉熵损失的函数原型为sparse_softmax_cross_entropy_with_logits(_sential, labels, logdits, name)
#它与softmax_cross_entropy_with_logits()函数的计算方式相同，适用于每个类别相互独立且排斥的情况，
#即一幅图只能属于一类。在1.0.0版本的TensorFlow中，这个函数只能通过命名参数的方式来使用，在这里
#logits参数是神经网络不包括softmax层的前向传播结果，lables参数给出了训练数据的正确答案
cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=y,labels=tf.argmax(y_, 1))
#argmax()函数原型为argmax(input, axis, name, dimension),用于计算每一个样例的预测答案，
#其中input参数y是一个batch_size * 10(batch_size行，10列)的二维数组，每一行表示一个样例前向传播的结果，
#axis参数“1”表示选取最大值的操作仅在第一个维度中进行，即只在每一行选取最大值对应的下标。
#于是得到的结果是一个长度为batch_size的一维数组，这个一维数组中的值就表示了每一个样例对应的
#数字识别结果。


regularizer = tf.contrib.layers.l2_regularizer(0.0001)       #计算L2正则化损失函数
regularization = regularizer(weights1)+regularizer(weights2) #计算模型的正则化损失
loss = tf.reduce_mean(cross_entropy)+regularization          #总损失


#用指数衰减法设置学习率，这里staircase参数采用默认的False，即学习率连续衰减
laerning_rate = tf.train.exponential_decay(learning_rate,training_step,mnist.train.num_examples/batch_size,
                                                                                       learning_rate_decay)
#使用GradientDescentOptimizer优化算法来优化交叉熵损失和正则化损失
train_step= tf.train.GradientDescentOptimizer(learning_rate).minimize(loss,global_step=training_step)


#在训练这个模型时，每过一遍数据既需要通过反向传播来更新神经网络中的参数又需要
#更新每一个参数的滑动平均值，control_dependencies()用于完成这样的一次性多次操作，
# 同样的操作也可以使用下面这行代码完成：
# train_op = tf.group(train_step,averages_op)
with tf.control_dependencies([train_step,averages_op]):
     train_op = tf.no_op(name="train")


#检查使用了滑动平均值模型的神经网络前向传播结果是否正确。
#equal()函数原型为equal(x, y, name)，用于判断两个张量的每一维是否相等，如果相等返回True,否则返回False。
crorent_predicition = tf.equal(tf.argmax(average_y,1),tf.argmax(y_,1))

#cast()函数原型为cast(x, DstT, name)。在这里用于将一个bool型的数据转为float32类型
#之后会将得到的float32 的数据求一个平均值，这个平均值就是模型在这一组数据上的正确率
accuracy = tf.reduce_mean(tf.cast(crorent_predicition,tf.float32))


with tf.Session() as sess:
    #在稍早的版本中一般使用 initialize_all_variables()函数初始化全部变量
    tf.global_variables_initializer().run()

    #准备验证数据，
    validate_feed = {x:mnist.validation.images,y_:mnist.validation.labels}
    #准备测试数据，
    test_feed = {x:mnist.test.images,y_:mnist.test.labels}

    for i in range(max_steps):
        if i%1000==0:
            #计算滑动平均模型在验证数据上的结果。
            # 为了能得到百分数输出，需要将得到的validate_accuracy扩大100倍
            validate_accuracy = sess.run(accuracy, feed_dict=validate_feed)
            print("After %d trainging step(s) ,validation accuracy"
                  "using average model is %g%%"%(i,validate_accuracy*100))

        #产生这一轮使用的一个batch的训练数据，并进行训练
        #input_data.read_data_sets()函数生成的类提供了train.next_bacth()函数，
        #通过设置函数的batch_size参数就可以从所有的训练数据中提读取一小部分作为一个训练batch
        xs,ys = mnist.train.next_batch(batch_size=100)
        sess.run(train_op,feed_dict={x:xs,y_:ys})

    #使用测试数据集检验神经网络训练之后的最终正确率
    # 为了能得到百分数输出，需要将得到的test_accuracy扩大100倍
    test_accuracy = sess.run(accuracy,feed_dict=test_feed)
    print("After %d trainging step(s) ,test accuracy using average"
                  " model is %g%%"%(max_steps,test_accuracy*100))


W1010 11:11:23.389134 139800070674240 deprecation.py:323] From /home/liyuan3970/anaconda3/lib/python3.7/site-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:262: extract_images (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use tf.data to implement this functionality.


Extracting MNIST_data/train-images-idx3-ubyte.gz


W1010 11:11:23.988765 139800070674240 deprecation.py:323] From /home/liyuan3970/anaconda3/lib/python3.7/site-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:267: extract_labels (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use tf.data to implement this functionality.
W1010 11:11:24.017867 139800070674240 deprecation.py:323] From /home/liyuan3970/anaconda3/lib/python3.7/site-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:110: dense_to_one_hot (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use tf.one_hot on tensors.
W1010 11:11:24.139470 139800070674240 deprecation.py:323] From /home/liyuan3970/anaconda3/lib/python3.7/site-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:290: DataSet.__init__ (from tensorflow.contrib.learn.python.learn.

Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


W1010 11:11:24.888353 139800070674240 deprecation.py:323] From /home/liyuan3970/anaconda3/lib/python3.7/site-packages/tensorflow/python/training/moving_averages.py:433: Variable.initialized_value (from tensorflow.python.ops.variables) is deprecated and will be removed in a future version.
Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.


After 0 trainging step(s) ,validation accuracyusing average model is 12.02%
After 1000 trainging step(s) ,validation accuracyusing average model is 97.86%
After 2000 trainging step(s) ,validation accuracyusing average model is 98.28%
After 3000 trainging step(s) ,validation accuracyusing average model is 98.4%
After 4000 trainging step(s) ,validation accuracyusing average model is 98.5%
After 5000 trainging step(s) ,validation accuracyusing average model is 98.44%
After 6000 trainging step(s) ,validation accuracyusing average model is 98.42%
After 7000 trainging step(s) ,validation accuracyusing average model is 98.48%
After 8000 trainging step(s) ,validation accuracyusing average model is 98.52%
After 9000 trainging step(s) ,validation accuracyusing average model is 98.54%
After 10000 trainging step(s) ,validation accuracyusing average model is 98.52%
After 11000 trainging step(s) ,validation accuracyusing average model is 98.44%
After 12000 trainging step(s) ,validation accuracyusing

#### conv2d的用法

In [1]:
import tensorflow as tf
import numpy as np

#使用numpy工具初始化一个名为M的数组，形状为2x3，数据类型为float32
#并使用numpy的reshape()函数调整输入的格式
#注意，M不会被TensorFlow识别为张量
M = np.array([[[2],[1],[2],[-1]],[[0],[-1],[3],[0]],
              [[2],[1],[-1],[4]],[[-2],[0],[-3],[4]]],dtype="float32").reshape(1, 4, 4, 1)

#通过get_variable()函数创建过滤器的权重变量，上面介绍了卷积层
#这里声明的参数变量是一个四维矩阵，前面两个维度代表了过滤器的尺寸，
#第三个维度表示当前层的深度，第四个维度表示过滤器的深度。
filter_weight = tf.get_variable("weights",[2, 2, 1, 1],
    initializer = tf.constant_initializer([[-1, 4],[2, 1]]))

#通过get_variable()函数创建过滤器的偏置项，代码中[1]表示过滤器的深度。
#等于神经网络下一层的深度。
biases = tf.get_variable("biase", [1], initializer = tf.constant_initializer(1))


x = tf.placeholder('float32', [1,None, None,1])

#conv2d()函数实现了卷积层前向传播的算法。
#这个函数的第一个参数为当前层的输入矩阵，注意这个矩阵应该是一个四维矩阵，
#代表第一个维度的参数对应一个输入batch。如果在输入层，input[0, , , ]表示第一张图片，
#input[1, , , ]表示第二张图片，等等。函数第二个参数是卷积层的权重，第三个参数为
#不同维度上的步长。虽然第三个参数提供的是一个长度为4 的数组，
#但是第一个和最后一个数字要求一定是1，这是因为卷积层的步长只对矩阵的长和宽有效。
#最后一个参数是填充(padding的方法，有SAME或VALID 两种选择，
#其中SAME 表示添加全0填充，VALID表示不添加。
#函数原型conv2d(input,filter,strids,padding,us_cudnn_on_gpu_,data_format,name)
conv = tf.nn.conv2d(x, filter_weight, strides=[1, 1, 1, 1], padding="SAME")

#bias_add()函数具有给每一个节点加上偏置项点功能。这里不能直接使用加法的原因是
#矩阵上不同位置上的节点都需要加上同样的偏置项。因为过滤器深度为1，
#故偏置项只有一个数，结果为3x4的矩阵中每一个值都要加上这个偏置项。
#原型bias_add(value,bias,data_format,name)
add_bias = tf.nn.bias_add(conv, biases)

init_op=tf.global_variables_initializer()
with tf.Session() as sess:
    init_op.run()
    M_conv=sess.run(add_bias, feed_dict={x: M})

    #输出结果并不是一个张量，而是数组
    print("M after convolution: \n", M_conv)

M after convolution: 
 [[[[  2.]
   [  9.]
   [  1.]
   [  2.]]

  [[  2.]
   [ 15.]
   [  0.]
   [  9.]]

  [[ -1.]
   [ -7.]
   [ 16.]
   [  5.]]

  [[  3.]
   [-11.]
   [ 20.]
   [ -3.]]]]


##### pool的应用

In [ ]:
import tensorflow as tf
import numpy as np

M = np.array([[[-2],[2],[0],[3]],
              [[1],[2],[-1],[2]],
              [[0],[-1],[1],[0]]],dtype="float32").reshape(1, 3, 4, 1)
filter_weight = tf.get_variable("weights",[2, 2, 1, 1],
    initializer = tf.constant_initializer([[2, 0],[-1, 1]]))
biases = tf.get_variable('biases', [1], initializer = tf.constant_initializer(1))
x = tf.placeholder('float32', [1, None, None, 1])
conv = tf.nn.conv2d(x, filter_weight, strides=[1, 1, 1, 1], padding="SAME")
add_bias = tf.nn.bias_add(conv, biases)

#max_pool()函数实现了最大池化层的前向传播过程
#原型为max_pool(value,strides,padding,data_format,name)
#参数value为输入数据，strides为提供了步长信息，padding提供了是否使用全0填充。
pool = tf.nn.max_pool(add_bias, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding="SAME")

with tf.Session() as sess:
    tf.global_variables_initializer().run()
    M_conv = sess.run(add_bias, feed_dict={x: M})
    M_pool = sess.run(pool, feed_dict={x: M})
    print(" after average pooled: \n", M_pool)
    '''输出内容
    after average pooled:
    [[[[7.]
       [5.]]
      [[1.]
       [3.]]]]
    '''


#### 图片显示的方法


In [ ]:
import matplotlib.pyplot as plt
import tensorflow as tf

image = tf.gfile.FastGFile("/home/jiangziyang/images/duck.png", 'r').read()
with tf.Session() as sess:
    img_after_decode = tf.image.decode_png(image)

    # 函数原型central_crop(image,central_fraction)
    central_cropped = tf.image.central_crop(img_after_decode, 0.4)
    plt.imshow(central_cropped.eval())
    plt.show()

## 保存模型

In [1]:
import tensorflow as tf

#声明两个变量并计算其加和
a = tf.Variable(tf.constant([1.0,2.0],shape=[2]), name="a")
b = tf.Variable(tf.constant([3.0,4.0],shape=[2]), name="b")
result=a+b

#定义初始化全部变量的操作
init_op=tf.initialize_all_variables()
#定义Saver类对象用于保存模型
saver=tf.train.Saver()

with tf.Session() as sess:
    sess.run(init_op)
    # 模型保存到/home/jiangziyang/model路径下的model.ckpt文件，其中model是模型的名称
    saver.save(sess,"src/model/model.ckpt")
    # save函数的原型是
    # save(self,ses,save_path,global_step,latest_filename,meta_graph_suffix,
    #                                            write_meta_graph, write_state)

W1030 09:45:44.237374 139957896574784 deprecation.py:323] From /home/liyuan3970/anaconda3/lib/python3.7/site-packages/tensorflow/python/util/tf_should_use.py:193: initialize_all_variables (from tensorflow.python.ops.variables) is deprecated and will be removed after 2017-03-02.
Instructions for updating:
Use `tf.global_variables_initializer` instead.


## Mnist_train的训练模型保存与读取
### 保存

In [3]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data",one_hot=True)
batch_size = 100
learning_rate = 0.8
learning_rate_decay = 0.999
max_steps = 30000

#更改前向传播算法的定义,将得到权重参数和偏执参数的过程封装到了一个函数中
def hidden_layer(input_tensor,regularizer,name):
#要多多体会使用变量空间来管理变量的方便性。使用get_variable()函数会在训练神经
#网络时创建这些变量而在测试过程中通过保存的模型加载这些变量的取值，在测试过程中
#可以在加载变量时将滑动平均变量重命名，这样就会在测试过程中使用变量的滑动平均值
    with tf.variable_scope("hidden_layer"):
        weights = tf.get_variable("weights", [784, 500],
                         initializer=tf.truncated_normal_initializer(stddev=0.1))

        #如果调用该函数时传入了正则化的方法 ，那么在这里将参数求
        if regularizer!=None:
            tf.add_to_collection("losses",regularizer(weights))
        biases = tf.get_variable("biases", [500], initializer=tf.constant_initializer(0.0))
        hidden_layer = tf.nn.relu(tf.matmul(input_tensor, weights) + biases)

    with tf.variable_scope("hidden_layer_output"):
        weights = tf.get_variable("weights", [500, 10],
                                  initializer=tf.truncated_normal_initializer(stddev=0.1))
        if regularizer!=None:
            tf.add_to_collection("losses",regularizer(weights))
        biases = tf.get_variable("biases", [10], initializer=tf.constant_initializer(0.0))
        hidden_layer_output = tf.matmul(hidden_layer, weights) + biases
    return hidden_layer_output

#定义输出输出的部分没变
x = tf.placeholder(tf.float32, [None,784],name="x-input")
y_ = tf.placeholder(tf.float32, [None,10],name="y-output")

#定义L2正则化的办法被提前
regularizer = tf.contrib.layers.l2_regularizer(0.0001)

#将L2正则化的办法传入到hidden_layer()函数中
y = hidden_layer(x,regularizer,name="y")

training_step = tf.Variable(0,trainable=False)
averages_class = tf.train.ExponentialMovingAverage(0.99,training_step)
averages_op = averages_class.apply(tf.trainable_variables())

#不再定义average_y，因为average_y只在比较正确率时有用，
#在模型保存的程序中我们只输出损失
#average_y = hidden_layer(x,averages_class,name="average_y",reuse=True)

cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=y,
                                                 labels=tf.argmax(y_, 1)) #tf.argmax()
#计算总损失
loss = tf.reduce_mean(cross_entropy)+tf.add_n(tf.get_collection("losses"))

laerning_rate = tf.train.exponential_decay(learning_rate,training_step,
                mnist.train.num_examples/batch_size,learning_rate_decay)
train_step= tf.train.GradientDescentOptimizer(learning_rate).\
                                minimize(loss,global_step=training_step)

#也可以采用train_op = tf.group(train_step,averages_op)的形式
with tf.control_dependencies([train_step, averages_op]):
    train_op = tf.no_op(name="train")

#初始化Saver持久化类
saver=tf.train.Saver()
with tf.Session() as sess:
    tf.global_variables_initializer().run()

    #进行30000轮到训练
    for i in range(max_steps):
        x_train, y_train = mnist.train.next_batch(batch_size)
        _, loss_value, step = sess.run([train_op, loss, training_step],
                                          feed_dict={x: x_train, y_: y_train})

        #每隔1000轮训练就输出当前训练batch上的损失函数大小，并保存一次模型
        if i % 1000 == 0:
            print("After %d training step(s), loss on training batch is "
                                                    "%g." % (step, loss_value))
            #保存模型的时候给出了global_step参数，这样可以让每个模型文件都添加
            #代表了训练轮数的后缀，这样做的原因是方便检索
            saver.save(sess, "src/mnist_model/mnist_model.ckpt",
                                                         global_step=training_step)

W1030 09:57:24.432219 139957896574784 deprecation.py:323] From <ipython-input-3-1d9772b3ccec>:3: read_data_sets (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
W1030 09:57:24.434207 139957896574784 deprecation.py:323] From /home/liyuan3970/anaconda3/lib/python3.7/site-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:260: maybe_download (from tensorflow.contrib.learn.python.learn.datasets.base) is deprecated and will be removed in a future version.
Instructions for updating:
Please write your own downloading logic.
W1030 09:57:24.437143 139957896574784 deprecation.py:323] From /home/liyuan3970/anaconda3/lib/python3.7/site-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:262: extract_images (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed 

Extracting MNIST_data/train-images-idx3-ubyte.gz


W1030 09:57:24.760637 139957896574784 deprecation.py:323] From /home/liyuan3970/anaconda3/lib/python3.7/site-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:267: extract_labels (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use tf.data to implement this functionality.
W1030 09:57:24.778739 139957896574784 deprecation.py:323] From /home/liyuan3970/anaconda3/lib/python3.7/site-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:110: dense_to_one_hot (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use tf.one_hot on tensors.
W1030 09:57:24.923892 139957896574784 deprecation.py:323] From /home/liyuan3970/anaconda3/lib/python3.7/site-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:290: DataSet.__init__ (from tensorflow.contrib.learn.python.learn.

Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


W1030 09:57:25.286395 139957896574784 deprecation.py:323] From /home/liyuan3970/anaconda3/lib/python3.7/site-packages/tensorflow/python/training/moving_averages.py:433: Variable.initialized_value (from tensorflow.python.ops.variables) is deprecated and will be removed in a future version.
Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.


After 1 training step(s), loss on training batch is 2.83791.
After 1001 training step(s), loss on training batch is 0.264814.
After 2001 training step(s), loss on training batch is 0.163542.
After 3001 training step(s), loss on training batch is 0.143543.
After 4001 training step(s), loss on training batch is 0.11728.


W1030 09:58:01.813363 139957896574784 deprecation.py:323] From /home/liyuan3970/anaconda3/lib/python3.7/site-packages/tensorflow/python/training/saver.py:960: remove_checkpoint (from tensorflow.python.training.checkpoint_management) is deprecated and will be removed in a future version.
Instructions for updating:
Use standard file APIs to delete files with this prefix.


After 5001 training step(s), loss on training batch is 0.103275.
After 6001 training step(s), loss on training batch is 0.0902812.
After 7001 training step(s), loss on training batch is 0.0841291.
After 8001 training step(s), loss on training batch is 0.0823245.
After 9001 training step(s), loss on training batch is 0.070709.
After 10001 training step(s), loss on training batch is 0.068285.
After 11001 training step(s), loss on training batch is 0.0642364.
After 12001 training step(s), loss on training batch is 0.0639735.
After 13001 training step(s), loss on training batch is 0.0495205.
After 14001 training step(s), loss on training batch is 0.051955.
After 15001 training step(s), loss on training batch is 0.0476992.
After 16001 training step(s), loss on training batch is 0.0423945.
After 17001 training step(s), loss on training batch is 0.0389303.
After 18001 training step(s), loss on training batch is 0.0531002.
After 19001 training step(s), loss on training batch is 0.0395308.
Afte

## 模型加载和评估

In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

mnist = input_data.read_data_sets("MNIST_data", one_hot=True)


# 定义相同的前向传播过程，要保持命名空间和变量名的一致
def hidden_layer(input_tensor, regularizer, name):
    with tf.variable_scope("hidden_layer",reuse=tf.AUTO_REUSE):
        weights = tf.get_variable("weights", [784, 500],
                                  initializer=tf.truncated_normal_initializer(stddev=0.1))
        if regularizer != None:
            tf.add_to_collection("losses", regularizer(weights))
        biases = tf.get_variable("biases", [500], initializer=tf.constant_initializer(0.0))
        hidden_layer = tf.nn.relu(tf.matmul(input_tensor, weights) + biases)

    with tf.variable_scope("hidden_layer_output",reuse=tf.AUTO_REUSE):
        weights = tf.get_variable("weights", [500, 10],
                                  initializer=tf.truncated_normal_initializer(stddev=0.1))
        if regularizer != None:
            tf.add_to_collection("losses", regularizer(weights))
        biases = tf.get_variable("biases", [10], initializer=tf.constant_initializer(0.0))
        hidden_layer_output = tf.matmul(hidden_layer, weights) + biases
    return hidden_layer_output


x = tf.placeholder(tf.float32, [None, 784], name="x-input")
y_ = tf.placeholder(tf.float32, [None, 10], name="y-input")

# 因为测试时不必关注正则化损失的值，所以不会传入正则化的办法
y = hidden_layer(x, None, name="y")

# 计算正确率的过程也基本和第六章的样例一致
correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
variable_averages = tf.train.ExponentialMovingAverage(0.99)

# 通过变量重命名的方式加载模型，这里使用了滑动平均类提供的variables_to_restore()
# 于是就免去了在前向传播过程中调用求解滑动平均的函数来获取滑动平均值的过程
saver = tf.train.Saver(variable_averages.variables_to_restore())

with tf.Session() as sess:
    validate_feed = {x: mnist.validation.images, y_: mnist.validation.labels}
    test_feed = {x: mnist.test.images, y_: mnist.test.labels}

    # get_checkpoint_state()函数会通过checkpoint文件自动找到目录中最新模型的文件名
    # 函数原型get_checkpoint_state(checkpoint_dir,latest_filename)
    ckpt = tf.train.get_checkpoint_state("src/mnist_model/")

#     # 加载模型
    saver.restore(sess, ckpt.model_checkpoint_path)

    # 通过文件名得到模型保存时迭代的轮数
    global_step = ckpt.model_checkpoint_path.split('/')[-1].split('-')[-1]
    print("The latest ckpt is mnist_model.ckpt-%s" % (global_step))
    # 输出The latest ckpt is mnist_model.ckpt-29001

    # 计算在验证数据集上的准确率并打印出来
    accuracy_score = sess.run(accuracy, feed_dict=validate_feed)
    print("After %s training step(s), validation accuracy = %g%%"
          % (global_step, accuracy_score * 100))
    # 输出After 29001 training step(s), validation accuracy = 98.62%

    # 计算在测试数据集上的准确率并打印出来
    test_accuracy = sess.run(accuracy, feed_dict=test_feed)
    print("After %s trainging step(s) ,test accuracy = %g%%"
          % (global_step, test_accuracy * 100))
    # 输出After 29001 trainging step(s) ,test accuracy = 98.51%



W1030 10:20:33.400926 140244664342336 deprecation.py:323] From <ipython-input-1-6eee74d2e213>:4: read_data_sets (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
W1030 10:20:33.402638 140244664342336 deprecation.py:323] From /home/liyuan3970/anaconda3/lib/python3.7/site-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:260: maybe_download (from tensorflow.contrib.learn.python.learn.datasets.base) is deprecated and will be removed in a future version.
Instructions for updating:
Please write your own downloading logic.
W1030 10:20:33.404206 140244664342336 deprecation.py:323] From /home/liyuan3970/anaconda3/lib/python3.7/site-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:262: extract_images (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed 

Extracting MNIST_data/train-images-idx3-ubyte.gz


W1030 10:20:33.657950 140244664342336 deprecation.py:323] From /home/liyuan3970/anaconda3/lib/python3.7/site-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:267: extract_labels (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use tf.data to implement this functionality.
W1030 10:20:33.664761 140244664342336 deprecation.py:323] From /home/liyuan3970/anaconda3/lib/python3.7/site-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:110: dense_to_one_hot (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use tf.one_hot on tensors.
W1030 10:20:33.723299 140244664342336 deprecation.py:323] From /home/liyuan3970/anaconda3/lib/python3.7/site-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:290: DataSet.__init__ (from tensorflow.contrib.learn.python.learn.

Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


W1030 10:20:34.012470 140244664342336 deprecation.py:323] From /home/liyuan3970/anaconda3/lib/python3.7/site-packages/tensorflow/python/training/saver.py:1276: checkpoint_exists (from tensorflow.python.training.checkpoint_management) is deprecated and will be removed in a future version.
Instructions for updating:
Use standard file APIs to check for files with this prefix.


The latest ckpt is mnist_model.ckpt-29001
After 29001 training step(s), validation accuracy = 98.52%
After 29001 trainging step(s) ,test accuracy = 98.44%


## 保存模型为PB文件

In [2]:
import tensorflow as tf
#graph_util模块定义在tensorflow/python/framework/graph_util.py
from tensorflow.python.framework import graph_util

a = tf.Variable(tf.constant(1.0, shape=[1]), name="a")
b = tf.Variable(tf.constant(2.0, shape=[1]), name="b")
result = a + b
init_op = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init_op)

    # 导出主要记录了TensorFlow计算图上节点信息的GraphDef部分
    # 使用get_default_graph()函数获取默认的计算图
    graph_def = tf.get_default_graph().as_graph_def()

    # convert_variables_to_constants()函数表示用相同值的常量替换计算图中所有变量，
    # 原型convert_variables_to_constants(sess,input_graph_def,output_node_names,
    #                          variable_names_whitelist, variable_names_blacklist)
    # 其中sess是会话，input_graph_def是具有节点的GraphDef对象，output_node_names
    # 是要保存的计算图中的计算节点的名称，通常为字符串列表的形式，variable_names_whitelist
    # 是要转换为常量的变量名称集合(默认情况下，所有变量都将被转换)，
    # variable_names_blacklist是要省略转换为常量的变量名的集合。
    output_graph_def = graph_util.convert_variables_to_constants(sess, graph_def, ['add'])

    # 将导出的模型存入.pb文件
    with tf.gfile.GFile("src/pb/model.pb", "wb") as f:
    # SerializeToString()函数用于将获取到的数据取出存到一个string对象中，
    # 然后再以二进制流的方式将其写入到磁盘文件中
        f.write(output_graph_def.SerializeToString())

W1030 10:28:48.187401 140244664342336 deprecation.py:323] From <ipython-input-2-715c03ad4a1a>:24: convert_variables_to_constants (from tensorflow.python.framework.graph_util_impl) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.compat.v1.graph_util.convert_variables_to_constants`
W1030 10:28:48.188592 140244664342336 deprecation.py:323] From /home/liyuan3970/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/graph_util_impl.py:270: extract_sub_graph (from tensorflow.python.framework.graph_util_impl) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.compat.v1.graph_util.extract_sub_graph`


## 模型加载

In [2]:
import tensorflow as tf
# gfile模块定义在tensorflow/python/platform/gfile.py
# 包含GFile、FastGFile和Open三个没有线程锁定的文件I/O包装器类
from tensorflow.python.platform import gfile

with tf.Session() as sess:
    # 使用FsatGFile类的构造函数返回一个FastGFile类
    with gfile.FastGFile("src/pb/model.pb", 'rb') as f:
        graph_def = tf.GraphDef()
        # 使用FastGFile类的read()函数读取保存的模型文件，并以字符串形式
        # 返回文件的内容，之后通过ParseFromString()函数解析文件的内容
        graph_def.ParseFromString(f.read())

    # 使用import_graph_def()函数将graph_def中保存的计算图加载到当前图中
    # 原型import_graph_def(graph_def,input_map,return_elements,name,op_dict,
    #                                                     producer_op_list)
    result = tf.import_graph_def(graph_def, return_elements=["add:0"])

    print(sess.run(result))
    # 输出为[array([3.], dtype=float32)]

[array([3.], dtype=float32)]
